In [1]:
import tensorflow as tf

## Creating a Variable

In [2]:
my_variable = tf.get_variable("my_variable", [1, 2, 3])

In [3]:
my_int_variable = tf.get_variable(
    "my_int_variable", 
    [1, 2, 3], 
    dtype=tf.int32, 
    initializer=tf.zeros_initializer)

In [4]:
other_variable = tf.get_variable(
    "other_variable", 
    dtype=tf.int32, 
    initializer=tf.constant([23, 42]))

### Variable collections

In [5]:
my_local = tf.get_variable(
    "my_local", 
    shape=(), 
    collections=[tf.GraphKeys.LOCAL_VARIABLES])

In [6]:
my_non_trainable = tf.get_variable(
    "my_non_trainable", 
    shape=(), 
    trainable=False)

In [7]:
tf.add_to_collection("my_collection_name", my_local)

In [8]:
tf.get_collection("my_collection_name")

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]

### Device placement

In [9]:
# with tf.device("/device:GPU:1"):
#     v = tf.get_variable("v", [1])

In [10]:
# cluster_spec = {
#     "ps": ["ps0:2222", "ps1:2222"],
#     "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]
# }
# with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
#     v = tf.get_variable("v", shape=[20, 20])

## Initializing variables

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
sess.run(my_variable.initializer)

In [13]:
print(sess.run(tf.report_uninitialized_variables()))

[b'my_local']


In [14]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = tf.get_variable("w", initializer=v.initialized_value() + 1)

## Using variables

In [15]:
w = v + 1

In [16]:
assignment = v.assign_add(1)
sess.run(tf.global_variables_initializer())
sess.run(assignment)

1.0

In [17]:
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
    w = v.read_value()

## Sharing variables

In [18]:
def conv_relu(input, kernel_shape, bias_shape):
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

In [19]:
input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])

In [20]:
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        return conv_relu(relu1, [5, 5, 32, 32], [32])

In [21]:
with tf.variable_scope("model"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model", reuse=True):
    output2 = my_image_filter(input2)